In [1]:
import numpy as np
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read excel file with sheet name
dict_df = pd.read_excel('RFM_Dataset.xlsx', sheet_name= ["Orders", "Return","Segmentation"])


In [3]:
dict_df

{'Orders':       Row ID        Order ID Order Date  Ship Date       Ship Mode  \
 0        646  CA-2017-126221 2017-12-30 2018-01-05  Standard Class   
 1        907  CA-2017-143259 2017-12-30 2018-01-03  Standard Class   
 2        908  CA-2017-143259 2017-12-30 2018-01-03  Standard Class   
 3        909  CA-2017-143259 2017-12-30 2018-01-03  Standard Class   
 4       1297  CA-2017-115427 2017-12-30 2018-01-03  Standard Class   
 ...      ...             ...        ...        ...             ...   
 9989    1760  CA-2014-141817 2014-01-05 2014-01-12  Standard Class   
 9990     740  CA-2014-112326 2014-01-04 2014-01-08  Standard Class   
 9991     741  CA-2014-112326 2014-01-04 2014-01-08  Standard Class   
 9992     742  CA-2014-112326 2014-01-04 2014-01-08  Standard Class   
 9993    7981  CA-2014-103800 2014-01-03 2014-01-07  Standard Class   
 
      Customer ID      Channel  Postal Code       Product ID    Sales  \
 0       CC-12430  Home Office        47201  OFF-AP-10002457  2

In [4]:
# Get DataFrame from Dict
orders = dict_df.get('Orders')
returns = dict_df.get('Return')
segments = dict_df.get('Segmentation')

In [5]:
# information of each dataframe 
print(orders.info())
print(returns.info())
print(segments.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Row ID       9994 non-null   int64         
 1   Order ID     9994 non-null   object        
 2   Order Date   9994 non-null   datetime64[ns]
 3   Ship Date    9994 non-null   datetime64[ns]
 4   Ship Mode    9994 non-null   object        
 5   Customer ID  9994 non-null   object        
 6   Channel      9994 non-null   object        
 7   Postal Code  9994 non-null   int64         
 8   Product ID   9994 non-null   object        
 9   Sales        9994 non-null   float64       
 10  Quantity     9994 non-null   int64         
 11  Unit Cost    9994 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int64(3), object(5)
memory usage: 937.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 2 columns):
 #   Column    

In [6]:
orders.head(10)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Channel,Postal Code,Product ID,Sales,Quantity,Unit Cost
0,646,CA-2017-126221,2017-12-30,2018-01-05,Standard Class,CC-12430,Home Office,47201,OFF-AP-10002457,209.300,2,76.3945
1,907,CA-2017-143259,2017-12-30,2018-01-03,Standard Class,PO-18865,Consumer,10009,FUR-BO-10003441,323.136,4,77.7546
2,908,CA-2017-143259,2017-12-30,2018-01-03,Standard Class,PO-18865,Consumer,10009,TEC-PH-10004774,90.930,7,12.6003
3,909,CA-2017-143259,2017-12-30,2018-01-03,Standard Class,PO-18865,Consumer,10009,OFF-BI-10003684,52.776,3,10.9950
4,1297,CA-2017-115427,2017-12-30,2018-01-03,Standard Class,EB-13975,Corporate,94533,OFF-BI-10002103,13.904,2,4.6926
5,1298,CA-2017-115427,2017-12-30,2018-01-03,Standard Class,EB-13975,Corporate,94533,OFF-BI-10004632,20.720,2,7.1225
6,5092,CA-2017-156720,2017-12-30,2018-01-03,Standard Class,JM-15580,Consumer,80538,OFF-FA-10003472,3.024,3,1.2096
7,1878,CA-2017-118885,2017-12-29,2018-01-02,Standard Class,JG-15160,Consumer,90049,FUR-CH-10002880,393.568,4,109.4611
8,1879,CA-2017-118885,2017-12-29,2018-01-02,Standard Class,JG-15160,Consumer,90049,TEC-PH-10002563,302.376,3,93.2326
9,2875,US-2017-102638,2017-12-29,2017-12-31,First Class,MC-17845,Consumer,10035,OFF-FA-10002988,6.030,3,1.0251


In [7]:
#Remove the returned orders from the orders table.

df_not_return = orders[~orders['Order ID'].isin(returns[returns['Returned'] == 'Yes']['Order ID'])]

df_not_return.shape

(9194, 12)

In [8]:
# Remove duplicate rows from the orders DataFrame
orders.drop_duplicates(inplace=True)

In [9]:
#Calculating RFM of each customer

recent_date = pd.to_datetime("2017-12-31")

#create new dataframe concluding R, F, M group by each customer ID
df_rfm = df_not_return.groupby('Customer ID', as_index=False).\
            agg({'Order Date' : lambda date: (recent_date - date.max()).days,
                 'Order ID' : 'nunique',
                 'Sales' : 'sum'})

# Change the name of columns from dataframe df_rfm
df_rfm.columns = ['Customer ID','recency', 'frequency', 'monetary']
 
df_rfm

,Customer ID,recency,frequency,monetary
0,AA-10315,185,5,5563.560
1,AA-10375,20,9,1056.390
2,AA-10480,260,4,1790.512
3,AA-10645,483,5,5073.975
4,AB-10015,416,3,886.156
...,...,...,...,...
786,XP-21865,83,9,2110.726
787,YC-21895,5,4,5438.650
788,YS-21880,10,8,6720.444
789,ZC-21910,55,12,7892.998


In [10]:
score_recency = [5,4,3,2,1]
score = [1,2,3,4,5]

#Create a new column to store the quintile score of recency
df_rfm['recency_score'] = pd.qcut(df_rfm.recency, q=5, labels = score_recency)

#Create a new column to store the quintile score of frequency
df_rfm['frequency_score'] = pd.qcut(df_rfm.frequency, q=5, labels = score)

#Create a new column to store the quintile score of monetary
df_rfm['monetary_score'] = pd.qcut(df_rfm.monetary, q=5, labels = score)

df_rfm

,Customer ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score
0,AA-10315,185,5,5563.560,2,2,5
1,AA-10375,20,9,1056.390,5,5,2
2,AA-10480,260,4,1790.512,2,1,3
3,AA-10645,483,5,5073.975,1,2,5
4,AB-10015,416,3,886.156,1,1,2
...,...,...,...,...,...,...,...
786,XP-21865,83,9,2110.726,3,5,3
787,YC-21895,5,4,5438.650,5,1,5
788,YS-21880,10,8,6720.444,5,4,5
789,ZC-21910,55,12,7892.998,4,5,5


In [11]:
#Concatnation r,f,m into rfm column as string type
df_rfm['rfm'] = df_rfm['recency_score'].astype(str) +\
                df_rfm['frequency_score'].astype(str) + \
                df_rfm['monetary_score'].astype(str)

df_rfm['rfm'] = df_rfm['rfm'].astype(int)

df_rfm

,Customer ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,rfm
0,AA-10315,185,5,5563.560,2,2,5,225
1,AA-10375,20,9,1056.390,5,5,2,552
2,AA-10480,260,4,1790.512,2,1,3,213
3,AA-10645,483,5,5073.975,1,2,5,125
4,AB-10015,416,3,886.156,1,1,2,112
...,...,...,...,...,...,...,...,...
786,XP-21865,83,9,2110.726,3,5,3,353
787,YC-21895,5,4,5438.650,5,1,5,515
788,YS-21880,10,8,6720.444,5,4,5,545
789,ZC-21910,55,12,7892.998,4,5,5,455


In [12]:
rfm_segment_map = {}

for row in segments.itertuples(index=False):
    segment = row.Segment
    rfm_score_ranges = [
        int(score.strip())
        for score in row._1.split(',')
        ]
    for rfm_score_range in rfm_score_ranges:
        rfm_segment_map[rfm_score_range] = segment

rfm_segment_map

{555: 'Champions',
 554: 'Champions',
 544: 'Champions',
 545: 'Champions',
 454: 'Champions',
 455: 'Champions',
 445: 'Champions',
 543: 'Loyal',
 444: 'Loyal',
 435: 'Loyal',
 355: 'Loyal',
 354: 'Loyal',
 345: 'Loyal',
 344: 'Loyal',
 335: 'Loyal',
 553: 'Potential Loyalist',
 551: 'Potential Loyalist',
 552: 'Potential Loyalist',
 541: 'Potential Loyalist',
 542: 'Potential Loyalist',
 533: 'Potential Loyalist',
 532: 'Potential Loyalist',
 531: 'Potential Loyalist',
 452: 'Potential Loyalist',
 451: 'Potential Loyalist',
 442: 'Potential Loyalist',
 441: 'Potential Loyalist',
 431: 'Potential Loyalist',
 453: 'Potential Loyalist',
 433: 'Potential Loyalist',
 432: 'Potential Loyalist',
 423: 'Potential Loyalist',
 353: 'Potential Loyalist',
 352: 'Potential Loyalist',
 351: 'Potential Loyalist',
 342: 'Potential Loyalist',
 341: 'Potential Loyalist',
 333: 'Potential Loyalist',
 323: 'Potential Loyalist',
 512: 'New Customers',
 511: 'New Customers',
 422: 'New Customers',
 421: 

In [13]:
segment_new = pd.DataFrame.from_dict(rfm_segment_map, orient="index", columns=["Segment"])

segment_new = segment_new.reset_index() # reset index to move dictionary keys to a column
segment_new = segment_new.rename(columns={'index': 'rfm'}) # rename column to 'rfm'

segment_new

,rfm,Segment
0,555,Champions
1,554,Champions
2,544,Champions
3,545,Champions
4,454,Champions
...,...,...
120,112,Lost customers
121,121,Lost customers
122,131,Lost customers
123,141,Lost customers
